In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
import calendar
from global_land_mask import globe
import glob
import math
from scipy import stats
import os
import netCDF4 as nc

from gcm_loading.highres_read import read_var_hires
from gcm_loading.myReadGCMsDaily import read_var_mod
from gcm_loading.read_amip import read_amip_var
from regrid_wght_3d import regrid_wght_wnans

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
plt.clf()
plt.rcParams['figure.figsize'] = (10, 10)

<Figure size 600x400 with 0 Axes>

In [2]:
from con_models import get_cons
con, use_colors, varname, pvarname, modname, warming_modname, hiresmd, amip_md = get_cons()

In [3]:
#latitude range
latr1 = 30
latr2 = 80


In [4]:
time1=[2010, 1, 1]
time2=[2012, 12, 30]

lats_edges = np.arange(latr1,latr2+1,5)
lons_edges = np.arange(-180,181,5)

#binning
n_bins  = 20
max_c   = 500


In [5]:
for j in range(0,1): #len(modname)
    print('\n',modname[j])
    for i in varname:
        print(i)
        locals()[i+'__'+str(j+1)] = read_var_mod('surface', modname[j], 'historical', i, time1, time2)

    for k in pvarname:
        print(k)
        locals()[k+'__'+str(j+1)] = read_var_mod('p_level', modname[j], 'historical', k, time1, time2)

print('historical done')


 CESM2
sfcWind
/glade/scratch/geethma/cmip6/surface/CESM2/CMIP6.CMIP.NCAR.CESM2.historical.r11i1p1f1.day.sfcWind.gn.v20190514.sfcWind_day_CESM2_historical_r11i1p1f1_gn_20100101-20150101.nc


/glade/work/geethma/research/codes/gcm_loading/myReadGCMsDaily.py:73: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  data.extend(np.array(datai[ind1:ind2+1,:,:]))


ts
/glade/scratch/geethma/cmip6/surface/CESM2/CMIP6.CMIP.NCAR.CESM2.historical.r11i1p1f1.Eday.ts.gn.v20190514.ts_Eday_CESM2_historical_r11i1p1f1_gn_20100101-20150101.nc
psl
/glade/scratch/geethma/cmip6/surface/CESM2/CMIP6.CMIP.NCAR.CESM2.historical.r11i1p1f1.day.psl.gn.v20190514.psl_day_CESM2_historical_r11i1p1f1_gn_20100101-20150101.nc
ta
/glade/scratch/geethma/cmip6/p_level/CESM2/CMIP6.CMIP.NCAR.CESM2.historical.r11i1p1f1.day.ta.gn.v20190514.ta_day_CESM2_historical_r11i1p1f1_gn_20100101-20150101.nc
historical done


In [13]:
mm = len(modname)
for j in range(0,len(hiresmd)):
    print('\n',hiresmd[j])
    for i in varname:
        locals()[i+'__'+str(j+1+mm)] = read_var_hires('surface', hiresmd[j], 'highresSST-present', i, time1, time2)

    for k in pvarname:
        locals()[k+'__'+str(j+1+mm)] = read_var_hires('p_level', hiresmd[j], 'highresSST-present', k, time1, time2)

print('hires done')

FGOALS-f3-L
sfcWind
length files: 1
tas
length files: 1
psl
length files: 1
ta
length files: 3
BCC-CSM2-HR
sfcWind
length files: 3
tas
length files: 4
psl
length files: 4
ta
length files: 16
MRI-AGCM3-2-H
sfcWind
length files: 1
tas
length files: 1
psl
length files: 1
ta
length files: 3
MRI-AGCM3-2-S
sfcWind
length files: 3
tas
length files: 3
psl
length files: 3
ta
length files: 38


In [ ]:
mmm = len(modname)+len(hiresmd)
j = 2
print('\n',amip_md[j])
# def read_amip_var(level, modn, exper, varnm, time1, time2):
for i in varname:
    if i!='ts':
        locals()[i+'__'+str(1+mmm)] = read_amip_var('surface', amip_md[j], 'amip', i, time1, time2)

for k in pvarname:
    locals()[k+'__'+str(1+mmm)] = read_amip_var('p_level', amip_md[j], 'amip', k, time1, time2)

print('amip done')


In [7]:
M_plot = []
W_plot = []
W_erro = []
bin_count = 1000
UL_pres = 850
M_range = (-30,20)


In [2]:

test_num = 0
for i in range(len(all_models)-1,len(all_models)): #l,mm+len(hiresmd)
    print(i)
    if i<len(modname):
        print(modname[i],str(i))
    elif i==mmm:
        print(amip_md[2])
    else:
        print(hiresmd[i-mm],str(i))

    for j in varname:
        lat  = locals()[j+'__'+str(i+1)][0]
        lon  = locals()[j+'__'+str(i+1)][1]
        time = locals()[j+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        #print(j)
        locals()[j+str(i+1)] = locals()[j+'__'+str(i+1)][4]
        locals()[j+str(i+1)] = np.ma.filled(locals()[j+str(i+1)], fill_value=np.nan)
        locals()['plot_'+j+str(i+1)] = np.array(np.multiply(maskm,locals()[j+str(i+1)][:,lat_ind1[0]:lat_ind2[0],:]))
        locals()['grid_'+j+str(i+1)] = regrid_wght_wnans(lats,lon,locals()['plot_'+j+str(i+1)],lats_edges,lons_edges)[0]



    for k in pvarname:
        #print(k)
        lat  = locals()[k+'__'+str(i+1)][0]
        lon  = locals()[k+'__'+str(i+1)][1]
        time = locals()[k+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        locals()['plot_levels'+str(i+1)] = locals()['ta__'+str(i+1)][3]
        locals()['grid_'+k+str(i+1)] = []

        levels = locals()['plot_levels'+str(i+1)]

        for p in range(len(levels)):
            if levels[p] == UL_pres*100:
                print(levels[p])
                locals()[k+str(i+1)] = locals()[k+'__'+str(i+1)][4]
                locals()[k+str(i+1)] = np.ma.filled(locals()[k+str(i+1)], fill_value=np.nan)
                temp_700   = np.array(np.multiply(maskm,locals()[k+str(i+1)][:,p,lat_ind1[0]:lat_ind2[0],:]))
                grid_t_700 = regrid_wght_wnans(lats,lon,temp_700,lats_edges,lons_edges)[0]
                break;

    theta_700 = grid_t_700*(100000/ (UL_pres*100))**con
    theta_t2m = locals()['grid_ts'+str(i+1)]*(100000/locals()['grid_psl'+str(i+1)])**con

    t = min(len(theta_t2m),len(theta_700))
    M_700  = theta_t2m[0:t,:,:] - theta_700[0:t,:,:]
    plot_M = M_700.flatten()
    try_W  = locals()['grid_sfcWind'+str(i+1)][0:t,:,:]
    plot_W = try_W.flatten()

    ind = np.argsort(plot_M)

    final_M = np.sort(plot_M)
    final_W = plot_W[ind]

    indx = np.isnan(final_M*final_W)==False

    bin_means, bin_edges, binnumber       = stats.binned_statistic(final_M[indx], final_W[indx], 'mean', bins=n_bins,range=M_range)
    bin_means_c, bin_edges_c, binnumber_c = stats.binned_statistic(final_M[indx], final_W[indx], 'count', bins=n_bins,range=M_range)
    bin_means_s, bin_edges_s, binnumber_s = stats.binned_statistic(final_M[indx], final_W[indx], 'std', bins=n_bins,range=M_range)
    bin_means_x, bin_edges_x, binnumber_x = stats.binned_statistic(final_M[indx], final_M[indx], 'mean', bins=n_bins,range=M_range)

    ind_c = np.where(bin_means_c > max_c)

    std_err = bin_means_s/np.sqrt(bin_means_c)
    M_plot.append(np.ma.masked_invalid(bin_means_x[ind_c])) #[ind_c]
    W_plot.append(np.ma.masked_invalid(bin_means[ind_c]))
    W_erro.append(np.ma.masked_invalid(std_err[ind_c]))
    bin_count = min(bin_count,len(bin_means_c[ind_c]))

        

In [9]:
from obs_data_function import obs
merwind, macwind, temp, sfctemp, sfcpres, p_lev_obs, p_mer_lat, merlon, merlev, obs_time = obs('surface','T2M',  UL_pres, latr1, latr2)


OBSERVATIONS
(1091, 1127)
macwind done


In [10]:
grid_obs_wind     = regrid_wght_wnans(p_mer_lat,merlon,macwind,lats_edges,lons_edges)[0]
grid_obs_temp_700 = regrid_wght_wnans(p_mer_lat,merlon,temp,lats_edges,lons_edges)[0]
grid_obs_temp_sfc = regrid_wght_wnans(p_mer_lat,merlon,sfctemp,lats_edges,lons_edges)[0]
grid_obs_pres_sfc = regrid_wght_wnans(p_mer_lat,merlon,sfcpres,lats_edges,lons_edges)[0]

lat_n = regrid_wght_wnans(p_mer_lat,merlon,sfcpres,lats_edges,lons_edges)[2][:,0]
lon_n = regrid_wght_wnans(p_mer_lat,merlon,sfcpres,lats_edges,lons_edges)[1][0,:]

theta_700 = np.array(np.multiply(grid_obs_temp_700, (100000/(merlev[p_lev_obs]*100))**(con)))
theta_sfc = np.array(np.multiply(grid_obs_temp_sfc, (100000/grid_obs_pres_sfc)**(con)))
print('theta_800', np.shape(theta_700))
print('theta_sfc', np.shape(theta_sfc))


p_CAOI = np.array(np.subtract(theta_sfc,theta_700))



theta_800 (1055, 10, 72)
theta_sfc (1055, 10, 72)


In [11]:
#Mask for the ocean
maskm = np.ones((len(temp),len(lat_n),len(lon_n)))

for a in range(len(lat_n)):
    for b in range(len(lon_n)):
        if globe.is_land(lat_n[a], lon_n[b])==True:
            maskm[:,a,b] = math.nan
##############################


In [12]:
#ocean only mask
plot_CAOI = np.array(np.multiply(maskm,p_CAOI))
plot_wind = np.array(np.multiply(maskm,grid_obs_wind))

plot_indx = np.isnan(plot_CAOI*plot_wind)==False
plot_mer_theta = plot_CAOI[plot_indx]
plot_mac_wind  = plot_wind[plot_indx]

w_sfc = plot_mac_wind[plot_mac_wind>0]
m_700 = plot_mer_theta[plot_mac_wind>0]
###################################


In [13]:
from scipy import stats
bin_means, bin_edges, binnumber       = stats.binned_statistic(m_700, w_sfc, 'mean', bins=n_bins, range=M_range)
bin_means_c, bin_edges_c, binnumber_c = stats.binned_statistic(m_700, w_sfc, 'count', bins=n_bins, range=M_range)
bin_means_s, bin_edges_s, binnumber_s = stats.binned_statistic(m_700, w_sfc, 'std', bins=n_bins, range=M_range)
bin_means_x, bin_edges_x, binnumber_x = stats.binned_statistic(m_700, m_700, 'mean', bins=n_bins, range=M_range)

W_SFC = np.ma.masked_invalid(bin_means[bin_means_c>max_c])
M_700 = np.ma.masked_invalid(bin_means_x[bin_means_c>max_c])
s_err = bin_means_s[bin_means_c>max_c]/np.sqrt(bin_means_c[bin_means_c>max_c])
W_err = np.ma.masked_invalid(s_err)

In [ ]:
bin_count = min(bin_count, len(W_SFC))


In [ ]:
all_models = []
all_models.extend(modname)
all_models.extend(hiresmd)
all_models.append('GISS-E3-G')

In [ ]:
plt.clf()

for i in range(len(all_models)):
    plt.plot(M_plot[i][0:bin_count], W_plot[i][0:bin_count], label=all_models[i], color=use_colors[i], alpha=0.6)
    # plt.fill_between(M_plot[i][0:bin_count], W_plot[i][0:bin_count]-W_erro[i][0:bin_count], W_plot[i][0:bin_count]+W_erro[i][0:bin_count], color=use_colors[i], alpha=0.2)
    plt.annotate(xy=(M_plot[i][bin_count-1],W_plot[i][bin_count-1]), text=all_models[i],color=use_colors[i],fontsize=5)
    # print(i)
    

In [ ]:
plt.clf()

i = -1
plt.plot(M_plot[i][0:bin_count], W_plot[i][0:bin_count], label=all_models[i], color=use_colors[i])
# plt.fill_between(M_plot[i][0:bin_count], W_plot[i][0:bin_count]-W_erro[i][0:bin_count], W_plot[i][0:bin_count]+W_erro[i][0:bin_count], color=use_colors[i], alpha=0.2)
plt.annotate(xy=(M_plot[i][bin_count-1],W_plot[i][bin_count-1]), text=all_models[i],color=use_colors[i],fontsize=5)
    # print(i)
    

In [ ]:
# for i in range(mm,mm+len(hiresmd)):
#     plt.plot(M_plot[i][0:bin_count], W_plot[i][0:bin_count], label=hiresmd[i-mm], color=use_colors[i])
#     plt.fill_between(M_plot[i][0:bin_count], W_plot[i][0:bin_count]-W_erro[i][0:bin_count], W_plot[i][0:bin_count]+W_erro[i][0:bin_count], color=use_colors[i], alpha=0.2)
#     plt.annotate(xy=(M_plot[i][bin_count-1],W_plot[i][bin_count-1]), text=hiresmd[i-mm],color=use_colors[i],fontsize=5)
#     # print(i)
    

In [ ]:


plt.plot(M_700[0:bin_count], W_SFC[0:bin_count], color='black' ,linestyle='dashed',
     markersize=5,linewidth=2, label='Observations')
# plt.fill_between(M_700[0:bin_count], W_SFC[0:bin_count]-W_err[0:bin_count], W_SFC[0:bin_count]+W_err[0:bin_count], color='black' ,alpha=0.2)
plt.annotate(xy=(M_700[bin_count-1],W_SFC[bin_count-1]), text='Observations', color='black',fontsize=7)


# plt.fill_between([-8.6,-5.3],[5,5],[13,13],color='grey',alpha=0.2)
# plt.annotate(xy=(-8.3,12.5), text='Unstable CAOI',color='grey',fontsize=9)


In [ ]:
plt.ylabel('U10 [m/s]')
# plt.xlim(-20,0)
yti = str(UL_pres)
plt.xlabel(r"M ($\Theta_{t2m}$ - $\Theta_{"+yti+"})$ [K]")
plt.title('U10 vs M for oceans between '+str(latr1)+'N to '+str(latr2)+'N')
plt.savefig('../figures/GISS-E3-G_'+str(UL_pres)+'_t2m_U10vsM.png')


## M / U10 PDF

In [ ]:
plt.clf()
from scipy.stats import norm
import seaborn as sns

In [ ]:

test_num = test_num+10
for i in range(25,27): #l,mm+len(hiresmd)
    if i<len(modname):
        print(modname[i],str(i))
    else:
        print(hiresmd[i-mm],str(i))

    for j in varname:
        lat  = locals()[j+'__'+str(i+1)][0]
        lon  = locals()[j+'__'+str(i+1)][1]
        time = locals()[j+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        #print(j)
        locals()[j+str(i+1)] = locals()[j+'__'+str(i+1)][4]
        locals()[j+str(i+1)] = np.ma.filled(locals()[j+str(i+1)], fill_value=np.nan)
        locals()['plot_'+j+str(i+1)] = np.array(np.multiply(maskm,locals()[j+str(i+1)][:,lat_ind1[0]:lat_ind2[0],:]))
        locals()['grid_'+j+str(i+1)] = regrid_wght_wnans(lats,lon,locals()['plot_'+j+str(i+1)],lats_edges,lons_edges)[0]



    for k in pvarname:
        #print(k)
        lat  = locals()[k+'__'+str(i+1)][0]
        lon  = locals()[k+'__'+str(i+1)][1]
        time = locals()[k+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        locals()['plot_levels'+str(i+1)] = locals()['ta__'+str(i+1)][3]
        locals()['grid_'+k+str(i+1)] = []

        levels = locals()['plot_levels'+str(i+1)]

        for p in range(len(levels)):
            if levels[p] ==  UL_pres*100:
                #print(levels[p])
                locals()[k+str(i+1)] = locals()[k+'__'+str(i+1)][4]
                locals()[k+str(i+1)] = np.ma.filled(locals()[k+str(i+1)], fill_value=np.nan)
                temp_700   = np.array(np.multiply(maskm,locals()[k+str(i+1)][:,p,lat_ind1[0]:lat_ind2[0],:]))
                grid_t_700 = regrid_wght_wnans(lats,lon,temp_700,lats_edges,lons_edges)[0]
                break;

    theta_700 = grid_t_700*(100000/(UL_pres*100))**con
    theta_t2m = locals()['grid_tas'+str(i+1)]*(100000/locals()['grid_psl'+str(i+1)])**con

    t = min(len(theta_t2m),len(theta_700))
    M_700  = theta_t2m[0:t,:,:] - theta_700[0:t,:,:]
    plot_M = M_700.flatten()
    try_W  = locals()['grid_sfcWind'+str(i+1)][0:t,:,:]
    plot_W = try_W.flatten()

    ind = np.argsort(plot_M)

    final_M = np.sort(plot_M)
    final_W = plot_W[ind]

    indx = np.isnan(final_M*final_W)==False

    variableG = final_W[indx]  ######change W/M here
    hx = np.histogram(variableG,1000)
    xvals = hx[1][:-1]
    xvalsSmooth = np.linspace(np.min(xvals),np.max(xvals),100)
    kernel = stats.gaussian_kde(variableG)
    plt.plot(xvalsSmooth,kernel.pdf(xvalsSmooth),color=use_colors[i],alpha=0.5)
# plt.savefig('../figures/testpdf.png')
        

In [ ]:
variableO = w_sfc[np.isnan(w_sfc)==False]   ###change m_700 / w_sfc
test_yy_new = np.sort(variableO)

##############################################
hx = np.histogram(test_yy_new,1000)
xvals = hx[1][:-1]
xvalsSmooth = np.linspace(np.min(xvals),np.max(xvals),100)
kernel = stats.gaussian_kde(test_yy_new)
plt.plot(xvalsSmooth,kernel.pdf(xvalsSmooth),linestyle = '--', color = 'k')
##############################################
####change U10 / M
plt.title('U10 PDF')
plt.savefig('../figures/new_final_U10PDF.png')
